In [ ]:
import torch
import torch.nn as nn

from torch.utils.tensorboard import SummaryWriter

In [2]:
import sys
sys.path.append('/home/raquelpanadero/Desktop/TFG/scripts')

from utils import create_dataloaders
from models.bb_crops_encoder_decoder import BBCropsEncoderDecoder
from trainers.bb_crops_trainer import train_model

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
data = torch.load('/home/raquelpanadero/Desktop/TFG/waymo_dataset_images/dataset.pt')

## Training

In [ ]:
list_batch_sizes = [16]
list_lr = [0.01, 0.001, 0.0001, 1e-5, 1e-6]

# define the net
input_size = 8
output_size = 4
hidden_size = 512
num_layers = 1
drop_prob = 0.5
target_len = 9
n_epochs = 40
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for batch_size in list_batch_sizes:
    for lr in list_lr:
        train_loader, valid_loader, test_loader = create_dataloaders(data, batch_size=batch_size)
        writer = SummaryWriter(
            log_dir='/home/raquelpanadero/Desktop/TFG/logs'
                    f'/dinov2_lr_{lr}_bsize_{batch_size}'
        )
        net = BBCropsEncoderDecoder(
            input_size=input_size,
            output_size=output_size, 
            batch_size=batch_size, 
            target_len=target_len, 
            hidden_size=hidden_size,
            num_layers=num_layers,
            device=device,
            feat_extractor = 'dino',
            fine_tuning = False
        )
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)

        train_model(
            net = net,
            train_loader = train_loader,
            valid_loader = valid_loader,
            n_epochs = n_epochs, 
            optimizer = optimizer,
            criterion = criterion,
            device = device,
            writer=writer
        )

In [ ]:
batch_size = 16
lr = 0.001
models = ['dino', 'vgg', 'resnet']
finetuning = [True, False]
normalization_layers = [True, False]

# define the net
input_size = 8
output_size = 4
hidden_size = 512
num_layers = 1
drop_prob = 0.5
target_len = 9
n_epochs = 40
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for model in models:
    for ft in finetuning:
        for nlayer in normalization_layers:
            train_loader, valid_loader, test_loader = create_dataloaders(data, batch_size=batch_size)
            writer = SummaryWriter(
                log_dir='/home/raquelpanadero/Desktop/TFG/logs'
                        f'/{model}_finetuning={ft}_normlayer={nlayer}_e{n_epochs}_lr_{lr}_bsize_{batch_size}'
            )
            net = BBCropsEncoderDecoder(
                input_size=input_size,
                output_size=output_size, 
                batch_size=batch_size, 
                target_len=target_len, 
                hidden_size=hidden_size,
                num_layers=num_layers,
                device=device,
                feat_extractor = model,
                fine_tuning = ft,
                normalization_layers = True
            )
            criterion = nn.MSELoss()
            optimizer = torch.optim.Adam(net.parameters(), lr=lr)

            train_model(
                net = net,
                train_loader = train_loader,
                valid_loader = valid_loader,
                n_epochs = n_epochs, 
                optimizer = optimizer,
                criterion = criterion,
                device = device,
                writer=writer,
                checkpoints_path = f'/home/raquelpanadero/Desktop/TFG/trained_models/{model}_finetuning={ft}_normlayer={nlayer}'
            )

## Model evaluation

In [ ]:
%tensorboard --logdir /home/raquelpanadero/Desktop/TFG/logs